# Initial Setup

## reference topology 
<img src='ref_topo.png'>

## EIGRP Configuration

__Router 1__ <br>
` conf t 
  int f0/0
  ip address 12.0.0.1 255.255.255.252
  no shutdown
  int f0/1
  ip address 13.0.0.1 255.255.255.252
  no sh
  int lo1
  ip address 192.168.1.1 255.255.255.255
  exit
  router eigrp 100
  no auto-summary
  network 0.0.0.0
  do wr
  `
  
__Router 2__ <br>
` conf t
  int f0/0
  ip address 12.0.0.2 255.255.255.252
  no shutdown
  int f0/1
  ip address 23.0.0.1 255.255.255.252
  no sh
  int lo1
  ip address 192.168.2.1 255.255.255.255
  exit
  router eigrp 100
  no auto-summary
  network 0.0.0.0
  do wr
  `
  
__Router 3__ <br>
` conf t
  int f0/0
  ip address 13.0.0.2 255.255.255.252
  no shutdown
  int f0/1
  ip address 23.0.0.2 255.255.255.252
  no sh
  exit
  int lo1
  ip address 192.168.3.1 255.255.255.255
  router eigrp 100
  no auto-summary
  network 0.0.0.0
  do wr
  `

## Activating SSH

* make a NAT cloud connection through f1/0 interface
* interface address from DHCP
* interface is passive 
* login credential : rishi / password
* key len : 1024

<img src='ref_topo_ssh.png'>

` conf t
  int f1/0
  ip address dhcp
  no sh
  exit
  router eigrp 100
  passive interface f1/0
  exit
  ip domain name lsbu.ac.uk
  ip ssh version 2
  username rishi password password
  crypto key generate rsa mod 1024
  line vty 0 15
  login local
  transport input ssh
  transport output ssh
  exit
  do wr
  `

## setting up NAPALM

__installing NAPALM__ <br>
`
sudo apt -y update 
sudo apt -y install python3 build-essential libssl-dev libffidev python-pip 
sudo python3.6 -m pip install --upgrade pip
sudo python3.6 -m pip install cryptography netmiko napalm
`

In [1]:
import napalm as nm

## fetching data from Routers

if `init_conn()` stuggles to eshtablish connection, use the following commands to the routers<br>

`
conf t
ip ssh ver 2
crypto key gen rsa mod 1024
`<br>
this will generate a new key pair and (re)start ssh2 server

In [3]:
# specify the network device 

c_driver=nm.get_network_driver('ios')
router_ip_list=['192.168.182.131','192.168.182.134','192.168.182.135']
credentials=['rishi','password','lsbu']   #[un, pw, sec]

In [4]:
# make connection to devices
def init_conn(c_driver, router_ip_list, credentials):
    router_obj_list=[]
    for i in range(len(router_ip_list)):
        router_obj_list.append(c_driver(hostname=router_ip_list[i],
                                 username=credentials[0],
                                 password=credentials[1],
                                 optional_args={'secret':'lsbu'}
                               ))
        try:
            router_obj_list[-1].open()
            print(f'connection {i+1} Successful! ')
        except Exception as e:
            print(e)
            print(f'connection {i+1} Failed')
            
    return router_obj_list

r_obj_list=init_conn(c_driver, router_ip_list, credentials)

connection 1 Successful! 
connection 2 Successful! 
connection 3 Successful! 


In [5]:
import pandas as pd
import json as js

In [6]:
# receive facts of the routers 
def get_facts(r_obj_list):
    r_fact_dict={'vendor':[],
               'model':[],
               'hostname':[],
               'fqdn':[]
              }
        
    for ro in r_obj_list:
        temp=ro.get_facts()
        r_fact_dict['vendor'].append(temp['vendor'])
        r_fact_dict['model'].append(temp['model'])
        r_fact_dict['hostname'].append(temp['hostname'])
        r_fact_dict['fqdn'].append(temp['fqdn'])
    
    r_fact_df=pd.DataFrame(r_fact_dict)
    
    return r_fact_df

r_fact_df=get_facts(r_obj_list)

r_fact_df

,vendor,model,hostname,fqdn
0,Cisco,3660,R1,R1.lsbu.ac.uk
1,Cisco,3660,R2,R2.lsbu.ac.uk
2,Cisco,3660,R3,R3.lsbu.ac.uk


In [7]:
def get_int_details(router_obj):
    x=router_obj.get_interfaces_ip()
    y=r_obj_list[0].get_interfaces()

    int_l=list(x)
    proto=[]
    for i in int_l:
        proto.append(list(x[i].keys())[0])
    ip=[]
    for i in range(len(proto)):
        ip.append(list(x[int_l[i]][proto[i]].keys())[0])
    cidr=[]
    for i in range(len(ip)):
        cidr.append(x[int_l[i]][proto[i]][ip[i]]['prefix_length'])
    mac=[]
    speed=[]
    mtu=[]
    bw=[]
    dly=[]
    rly=[]
    ld=[]
    for i in int_l:
        mac.append(y[i]['mac_address'])
        speed.append(y[i]['speed'])
        
        mbd=list(router_obj.cli([f'sh int {i} | inc MTU']).values())[0]
        rl=list(router_obj.cli([f'sh int {i} | inc reliability']).values())[0]
        
        mtu.append(int(mbd.split(' ')[1]))
        bw.append(int(mbd.split(' ')[4]))
        dly.append(int(mbd.split(' ')[7]))
        rly.append(eval(rl.split(' ')[1])[0])
        ld.append(round((eval(rl.split(' ')[3].split(',')[0]) + eval(rl.split(' ')[5]))/2,3))
    
    
    return[int_l, proto, ip, cidr, mac, speed, mtu, bw, dly, rly, ld]

fetch the interface details of the first router object 

In [8]:
print(r_obj_list)
get_int_details(r_obj_list[0])

[<napalm.ios.ios.IOSDriver object at 0x0000027DB6AC12B0>, <napalm.ios.ios.IOSDriver object at 0x0000027DB8763BA8>, <napalm.ios.ios.IOSDriver object at 0x0000027DB87829E8>]


[['FastEthernet0/0', 'FastEthernet0/1', 'FastEthernet1/0', 'Loopback1'],
 ['ipv4', 'ipv4', 'ipv4', 'ipv4'],
 ['12.0.0.1', '13.0.0.1', '192.168.182.131', '192.168.1.1'],
 [30, 30, 24, 32],
 ['CC:01:39:18:00:00', 'CC:01:39:18:00:01', 'CC:01:39:18:00:10', ''],
 [100, 100, 100, 8000],
 [1500, 1500, 1500, 1514],
 [100000, 100000, 100000, 8000000],
 [100, 100, 100, 5000],
 [1.0, 1.0, 1.0, 1.0],
 [0.004, 0.004, 0.004, 0.004]]

In [10]:
def get_intf(r_obj_list):
    
    intf_dict={}
    ret=[]
    
    count=0
    for ro in r_obj_list:
        intf=get_int_details(ro)
        count+=1
        print(f'Fetch Successful... {count}/{len(r_obj_list)}')
        
        intf_dict['interface']=intf[0]
        intf_dict['protocol']=intf[1]
        intf_dict['IP']=intf[2]
        intf_dict['CIDR']=intf[3]
        intf_dict['MAC']=intf[4]
        intf_dict['speed']=intf[5]
        intf_dict['mtu']=intf[6]
        intf_dict['BW']=intf[7]
        intf_dict['delay']=intf[8]
        intf_dict['reliability']=intf[9]
        intf_dict['load']=intf[10]
        
        
        ret.append(pd.DataFrame(intf_dict))
    
    return {'r_name' : r_fact_df['hostname'].tolist() , 'r_details' : ret}

Test for all routers

In [11]:
from IPython.display import display, HTML

for i in get_intf(r_obj_list)['r_details']:
    display(i)

Fetch Successful... 1/3
Fetch Successful... 2/3
Fetch Successful... 3/3


,interface,protocol,IP,CIDR,MAC,speed,mtu,BW,delay,reliability,load
0,FastEthernet0/0,ipv4,12.0.0.1,30,CC:01:39:18:00:00,100,1500,100000,100,1.0,0.004
1,FastEthernet0/1,ipv4,13.0.0.1,30,CC:01:39:18:00:01,100,1500,100000,100,1.0,0.004
2,FastEthernet1/0,ipv4,192.168.182.131,24,CC:01:39:18:00:10,100,1500,100000,100,1.0,0.004
3,Loopback1,ipv4,192.168.1.1,32,,8000,1514,8000000,5000,1.0,0.004


,interface,protocol,IP,CIDR,MAC,speed,mtu,BW,delay,reliability,load
0,FastEthernet0/0,ipv4,12.0.0.2,30,CC:01:39:18:00:00,100,1500,100000,100,1.0,0.004
1,FastEthernet0/1,ipv4,23.0.0.1,30,CC:01:39:18:00:01,100,1500,100000,100,1.0,0.004
2,FastEthernet1/0,ipv4,192.168.182.134,24,CC:01:39:18:00:10,100,1500,100000,100,1.0,0.004
3,Loopback1,ipv4,192.168.2.1,32,,8000,1514,8000000,5000,1.0,0.004


,interface,protocol,IP,CIDR,MAC,speed,mtu,BW,delay,reliability,load
0,FastEthernet0/0,ipv4,13.0.0.2,30,CC:01:39:18:00:00,100,1500,100000,100,1.0,0.004
1,FastEthernet0/1,ipv4,23.0.0.2,30,CC:01:39:18:00:01,100,1500,100000,100,1.0,0.004
2,FastEthernet1/0,ipv4,192.168.182.135,24,CC:01:39:18:00:10,100,1500,100000,100,1.0,0.004


# gathering EIGRP Tables 

In [10]:
ro=r_obj_list[0]
x=list(ro.get_interfaces_ip())

mbd=list(ro.cli([f'sh int {x[0]} | inc MTU']).values())[0]
rl=list(ro.cli([f'sh int {x[0]} | inc reliability']).values())[0]
print(mbd)
print(rl)

MTU 1500 bytes, BW 100000 Kbit/sec, DLY 100 usec,
reliability 255/255, txload 1/255, rxload 1/255


In [11]:
mtu=int(mbd.split(' ')[1])
bw=int(mbd.split(' ')[4])
dly=int(mbd.split(' ')[7])
rly=eval(rl.split(' ')[1])[0]
ld=round((eval(rl.split(' ')[3].split(',')[0]) + eval(rl.split(' ')[5]))/2,3)

#round(eval(rl.split(' ')[3].split(',')[0]),3)
         
print(mtu, bw,dly,rly,ld)

1500 100000 100 1.0 0.004


In [12]:
r_fact_df['hostname'].tolist()

['R1', 'R2', 'R3']